# All methods of using tests

There are some things you can and some things you can't do with tests.  This tutorial will show all of them.

In [25]:
from nornir_tests.plugins.processors import TestsProcessor
from nornir_tests.plugins.tests import *
from nornir_napalm.plugins.tasks import napalm_get, napalm_ping
from nornir_utils.plugins.functions import print_result
from nornir import InitNornir

nr = InitNornir(
    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "data/hosts.yaml",
            "group_file": "data/groups.yaml",
            "defaults_file": "data/defaults.yaml",
        },
    },
)

nr.processors.append(TestsProcessor())

vyos = nr.filter(name="vyos")

## @ style decorator use

This will only work as designed if not using task.run inside the function.  If there were a bunch of task.run statements in the function the decorators applied using @ syntax would not get applied.

In [26]:
@test_until(retries=5, delay=5)
@test_timing(max_run_time=5, fail_task=True)
@test_jsonpath(path='interfaces.eth0.is_enabled', value=True, fail_task=True)
def at_syntax_test(task):
    return napalm_get(task, getters=['interfaces'])

print_result(vyos.run(task=at_syntax_test), vars=['tests'])

inner***************************************************************************
* vyos ** changed : False ******************************************************
vvvv inner ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
('jsonpath: interfaces.eth0.is_enabled expanded to interfaces.eth0.is_enabled '
 'and value True found - PASSED')
"('timing: 0 < 2.02677583694458 < 5',) - PASSED"
'until: succeeded after 13.71741509437561 seconds - PASSED'

^^^^ END inner ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Using nr.run with tasks that return results

In [27]:
print_result(vyos.run(
    task=napalm_get,
    getters=['interfaces'],
    tests=[
        test_timing(),
        test_jsonpath(path='interfaces.eth0.is_up', value=True)
    ]
), vars=['tests'])

napalm_get**********************************************************************
* vyos ** changed : False ******************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
"('timing: 0 < 2.010446071624756 < 9223372036854775807',) - PASSED"
('jsonpath: interfaces.eth0.is_up expanded to interfaces.eth0.is_up and value '
 'True found - PASSED')

^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Using task.run inside grouped task

In [28]:
def grouped_task(task):
    task.run(
        napalm_ping,
        dest='192.168.99.1',
        tests=[
            test_timing()
        ]
    )

    task.run(
        napalm_get, 
        getters=['interfaces'], 
        tests=[
            test_jsonpath(path='interfaces.eth0.is_up', value=True)
        ]
    )

print_result(vyos.run(task=grouped_task), vars=['tests'])

grouped_task********************************************************************
* vyos ** changed : False ******************************************************
vvvv grouped_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
---- napalm_ping ** changed : False -------------------------------------------- INFO
"('timing: 0 < 4.911297082901001 < 9223372036854775807',) - PASSED"

---- napalm_get ** changed : False --------------------------------------------- INFO
('jsonpath: interfaces.eth0.is_up expanded to interfaces.eth0.is_up and value '
 'True found - PASSED')

^^^^ END grouped_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Not supported

Testing a grouped_task at the level of running nr.run will not work.  There is not much you would be able to test without really complex logic anyhow considering this always returns an AggregatedResult.